In [1]:
import sys
sys.path.append(r"scripts")
from functools import lru_cache
import pandas as pd
import numpy as np
import torch
import torch

# Check if PyTorch is installed
print(f"PyTorch version: {torch.__version__}")

print(torch.version.cuda)  # Should match 12.6 or similar
print(torch.backends.cudnn.enabled)  # Sho
from torch_geometric.data import Data
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import os
import requests
import zipfile
import networkx as nx
import scripts
from scripts import *
import torchmetrics
from torch import nn
import optuna
import models
from optuna.integration import TensorBoardCallback
from model_GNN import ModularPathwayConv, ModularGNN
torch.set_printoptions(threshold=torch.inf)
from model_ResNet import CombinedModel, ResNet, DrugMLP  
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from copy import deepcopy
import itertools
print(f"Memory allocated: {torch.cuda.memory_allocated() / 1e6} MB")
print(f"Max memory allocated: {torch.cuda.max_memory_allocated() / 1e6} MB")
import uuid


PyTorch version: 2.5.1.post106
None
True
Memory allocated: 0.0 MB
Max memory allocated: 0.0 MB


In [2]:
@lru_cache(maxsize=None)
def get_data(n_fold=0, fp_radius=2):
    """Download, process, and prepare data for use in graph-based machine learning models."""
    import os
    import zipfile
    import requests
    import torch
    import pandas as pd
    import numpy as np
    import networkx as nx
    from torch_geometric.data import Data
    import scripts  # Assuming scripts has required functions

    def download_if_not_present(url, filepath):
        """Download a file from a URL if it does not exist locally."""
        if not os.path.exists(filepath):
            print(f"File not found at {filepath}. Downloading...")
            response = requests.get(url, stream=True)
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            with open(filepath, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            print("Download completed.")
        else:
            print(f"File already exists at {filepath}.")

    # Step 1: Download and load RNA-seq data
    zip_url = "https://cog.sanger.ac.uk/cmp/download/rnaseq_all_20220624.zip"
    zip_filepath = "data/rnaseq.zip"
    rnaseq_filepath = "data/rnaseq_normcount.csv"
    if not os.path.exists(rnaseq_filepath):
        download_if_not_present(zip_url, zip_filepath)
        with zipfile.ZipFile(zip_filepath, "r") as zipf:
            zipf.extractall("data/")
    rnaseq = pd.read_csv(rnaseq_filepath, index_col=0)

    # Step 2: Load gene network, hierarchies, and driver genes
    hierarchies = pd.read_csv("data/gene_to_pathway_final_with_hierarchy.csv")
    driver_genes = pd.read_csv("data/driver_genes_2.csv")['gene'].dropna()
    gene_network = nx.read_edgelist("data/filtered_gene_network.edgelist", nodetype=str)
    ensembl_to_hgnc = dict(zip(hierarchies['Ensembl_ID'], hierarchies['HGNC']))
    mapped_gene_network = nx.relabel_nodes(gene_network, ensembl_to_hgnc)

    # Step 3: Filter RNA-seq data and identify valid nodes
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    valid_nodes = set(filtered_rna.columns)  # Get valid nodes after filtering RNA-seq columns

    # Step 4: Create edge tensors for the graph
    edges_df = pd.DataFrame(
        list(mapped_gene_network.edges(data="weight")),
        columns=["source", "target", "weight"]
    )
    edges_df["weight"] = edges_df["weight"].fillna(1.0).astype(float)
    filtered_edges = edges_df[
        (edges_df["source"].isin(valid_nodes)) & (edges_df["target"].isin(valid_nodes))
    ]
    node_to_idx = {node: idx for idx, node in enumerate(valid_nodes)}
    filtered_edges["source_idx"] = filtered_edges["source"].map(node_to_idx)
    filtered_edges["target_idx"] = filtered_edges["target"].map(node_to_idx)
    edge_index = torch.tensor(filtered_edges[["source_idx", "target_idx"]].values.T, dtype=torch.long)
    edge_attr = torch.tensor(filtered_edges["weight"].values, dtype=torch.float32)

    # Step 5: Process the hierarchy to create pathway groups
    filtered_hierarchy = hierarchies[hierarchies["HGNC"].isin(valid_nodes)]
    pathway_dict = {
        gene: pathway.split(':', 1)[1].split('[', 1)[0].strip() if isinstance(pathway, str) and ':' in pathway else None
        for gene, pathway in zip(filtered_hierarchy['HGNC'], filtered_hierarchy['Level_1'])
    }
    grouped_pathway_dict = {}
    for gene, pathway in pathway_dict.items():
        if pathway:
            grouped_pathway_dict.setdefault(pathway, []).append(gene)
    pathway_groups = {
        pathway: [node_to_idx[gene] for gene in genes if gene in node_to_idx]
        for pathway, genes in grouped_pathway_dict.items()
    }
    # Convert to padded tensor
    pathway_tensors = pad_sequence(
        [torch.tensor(indices, dtype=torch.long) for indices in pathway_groups.values()], 
        batch_first=True, 
        padding_value=-1  # Use -1 as padding
    )

    # Step 6: Create cell-line graphs
    tensor_exp = torch.tensor(filtered_rna.to_numpy())
    cell_dict = {cell: tensor_exp[i] for i, cell in enumerate(filtered_rna.index.to_numpy())}
    graph_data_list = {}
    for cell, x in cell_dict.items():
        if x.ndim == 2 and x.shape[0] == 1:
            x = x.T
        elif x.ndim == 1:
            x = x.unsqueeze(1)
        graph_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
        graph_data.y = None
        graph_data.cell_line = cell
        graph_data_list[cell] = graph_data

    # Step 7: Load drug data
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R=fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])

    # Step 8: Load IC50 data and filter for valid cell lines and drugs
    data = pd.read_csv("data/GDSC1.csv", index_col=0)
    data = data.query("SANGER_MODEL_ID in @cell_dict.keys() & DRUG_ID in @drug_dict.keys()")

    # Step 9: Split the data into folds for cross-validation
    unique_cell_lines = data["SANGER_MODEL_ID"].unique()
    np.random.seed(420)
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold]

    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")

    # Step 10: Build the datasets for training, validation, and testing
    train_dataset = scripts.OmicsDataset(graph_data_list, drug_dict, train_data)
    validation_dataset = scripts.OmicsDataset(graph_data_list, drug_dict, validation_data)
    test_dataset = scripts.OmicsDataset(graph_data_list, drug_dict, test_data)

    return train_dataset, validation_dataset, test_dataset, pathway_tensors


In [3]:
train_data, val_data, test_data, pathway_groups=get_data(0)

/tmp/ipykernel_163826/3102354241.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edges["source_idx"] = filtered_edges["source"].map(node_to_idx)
/tmp/ipykernel_163826/3102354241.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edges["target_idx"] = filtered_edges["target"].map(node_to_idx)
[17:41:53] DEPRECATION WARNING: please use MorganGenerator
[17:41:53] DEPRECATION WARNING: please use MorganGenerator
[17:41:53] DEPRECATION WARNING: please use MorganGenerator
[17:41:53] DEPREC

In [4]:
def custom_collate_fn(batch):
    try:
        cell_graphs = [item[0] for item in batch if item[0] is not None]
        if len(cell_graphs) == 0:
            raise ValueError("No graphs to batch in this batch. Batch might be empty or contains None.")
        
        drug_vectors = torch.stack([item[1] for item in batch if item[1] is not None])  # Stack drug vectors
        targets = torch.stack([item[2] for item in batch if item[2] is not None])  # Stack target values
        cell_ids = torch.stack([item[3] for item in batch if item[3] is not None])  # Stack cell IDs
        drug_ids = torch.stack([item[4] for item in batch if item[4] is not None])  # Stack drug IDs

        # Batch the PyG graphs into a single DataBatch
        cell_graph_batch = Batch.from_data_list(cell_graphs)
        return cell_graph_batch, drug_vectors, targets, cell_ids, drug_ids
    
    except Exception as e:
        print(f"Error in custom_collate_fn: {e}")
        print(f"Batch contents: {batch}")
        raise e

In [5]:
#gnn_model = ModularGNN(**config["gnn"])
#drug_mlp = DrugMLP(input_dim=config["drug"]["input_dim"], embed_dim=config["drug"]["embed_dim"])
#resnet = ResNet(embed_dim=config["drug"]["embed_dim"], hidden_dim=config["resnet"]["hidden_dim"], 
#                n_layers=config["resnet"]["n_layers"], dropout=config["resnet"]["dropout"])
#
#combined_model = CombinedModel(gnn=gnn_model, drug_mlp=drug_mlp, resnet=resnet)
#
#device = torch.device(config["env"]["device"])
#combined_model.to(device)
print(pathway_groups)

tensor([[4440, 2589, 2834, 2478, 3571, 2747, 2954, 7538, 2485, 6276, 4530, 1471,
         3303, 1705, 5827, 5306, 4142, 2875, 2614, 5057, 7113, 7132, 4848, 1662,
         4548, 3689, 3615, 1757, 1137,  918, 6384, 3327, 4949, 4177, 7717, 2176,
          463, 1322, 1030, 5402, 1336, 6114, 4488, 1546, 4662, 4567, 6679, 1004,
         7694, 4297, 6590, 7086, 1264, 6177, 5074, 6389, 3717, 3997, 4666, 2358,
         7461, 5466, 1002,   31, 5988, 2877, 2106, 4267, 4575,  696, 5897, 7044,
          893, 1792, 4150, 3918, 5007, 5199, 3758, 6247, 3021, 2659, 1094, 2157,
         2593, 6726, 3937, 2083,  645, 4892, 6567,  654,   56, 1305, 2688, 2733,
          678, 6345, 5756, 1723, 1268, 4767, 7130, 1575, 1903, 6393, 1231, 2423,
         4872, 1050,  938,  827, 6955, 4558, 6837, 5709, 2509, 3264, 1461,  475,
         4863, 7225, 2156,  643, 4916, 2833, 6931, 5282, 7446, 5980, 3841, 6461,
         3735, 1376, 2638, 7508,  831, 3562, 3851, 3648, 7498, 6696, 4451, 7644,
         1026,  217, 2932, 7

In [6]:
def initialize_lazy_layers(model, test_instance, drug_input_size, device='cpu'):
    if hasattr(model, 'lazy_initialized') and model.lazy_initialized:
        print("Lazy layers are already initialized.")
        return  # Don't reinitialize

    print("Initializing lazy layers...")

    x = test_instance.x.to(device)  
    edge_index = test_instance.edge_index.to(device) 
    edge_attr = test_instance.edge_attr.to(device) if 'edge_attr' in test_instance else None
    pathway_tensor = test_instance.pathway_tensor.to(device) if 'pathway_tensor' in test_instance else None
    graph_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)  
    dummy_drug = torch.randn(1, drug_input_size).to(device)  
    
    with torch.no_grad():
        model(graph_data, dummy_drug)
    
    model.lazy_initialized = True  # Add an attribute to track initialization
    print("Lazy layers initialized with real instance.")
def train_model_optuna(trial, base_config, train_dataset, validation_dataset):

    copy_config = deepcopy(base_config)

    def pruning_callback(epoch, train_r):
        trial.report(train_r, step=epoch)
        if np.isnan(train_r):
            raise optuna.TrialPruned()
        if trial.should_prune():
            raise optuna.TrialPruned()

    
    AGGR_MODE_COMBINATIONS = [
        ('sum', 'mean', 'max'),
        ('mean', 'max', 'sum'),
        ('max', 'sum', 'mean'),
        ('sum', 'sum', 'sum'),
        ('mean', 'mean', 'mean'),
        ('max', 'max', 'max')
    ]

    copy_config["gnn"].update({
        "hidden_dim": trial.suggest_int("hidden_dim", 64, 512),
        "output_dim": trial.suggest_int("output_dim", 64, 256),  # Adjusted range to be large enough
        "pooling_mode": trial.suggest_categorical("pooling_mode", ["pathway", "scalar", "none"]),
        "aggr_modes": trial.suggest_categorical("aggr_modes", AGGR_MODE_COMBINATIONS)
    })

    pathway_count = 44  # Number of pathways
    gnn_output_dim = copy_config["gnn"]["output_dim"]

    # Handle pathway pooling
    if copy_config["gnn"]["pooling_mode"] == 'pathway':
        final_gnn_output_dim = pathway_count * gnn_output_dim 
    else:
        final_gnn_output_dim = gnn_output_dim

    copy_config["drug"].update({
        "input_dim": 2048,  
        "embed_dim": final_gnn_output_dim  # DrugMLP's embed_dim must match GNN's final output
    })

    copy_config["resnet"].update({
        "embed_dim": final_gnn_output_dim,  # ResNet now takes the size of pathway-aware embeddings
        "hidden_dim": trial.suggest_int("hidden_dim_resnet", 64, 512),
        "n_layers": trial.suggest_int("n_layers", 1, 6),
        "dropout": trial.suggest_float("dropout", 0.0, 0.5)
    })



    copy_config["optimizer"].update({
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True),
        "clip_norm": trial.suggest_float("clip_norm", 0.1, 20.0),
        "batch_size": trial.suggest_int("batch_size", 1, 8)
    })

    if not all(mode in ['sum', 'mean', 'max'] for mode in copy_config["gnn"]["aggr_modes"]):
        raise ValueError(f"Invalid aggregation mode selected: {copy_config['gnn']['aggr_modes']}")
    print(f"Using aggregation mode: {copy_config['gnn']['aggr_modes']}")
    try:

        gnn_model = ModularGNN(**copy_config["gnn"])  
        drug_mlp = DrugMLP(input_dim=copy_config["drug"]["input_dim"], embed_dim=copy_config["drug"]["embed_dim"])
        resnet = ResNet(
            embed_dim=copy_config["resnet"]["embed_dim"], 
            hidden_dim=copy_config["resnet"]["hidden_dim"], 
            n_layers=copy_config["resnet"]["n_layers"], 
            dropout=copy_config["resnet"]["dropout"]
        )
        
        combined_model = CombinedModel(gnn=gnn_model, drug_mlp=drug_mlp, resnet=resnet)
        train_loader = DataLoader(train_data, batch_size=1, shuffle=True) 
        test_instance = next(iter(train_loader)) 
        test_instance = test_instance[0] 
        initialize_lazy_layers(model=combined_model, 
                       test_instance=test_instance, 
                       drug_input_size=2048, 
                       device='cpu')  # Or device='cuda'


        device = torch.device(copy_config["env"]["device"])
        combined_model.to(device)
        print(device)
        if torch.cuda.device_count() > 1:
            combined_model = torch.nn.DataParallel(combined_model)

        R, model = train_model(copy_config, train_dataset, validation_dataset, callback_epoch=pruning_callback)
        return R

    except Exception as e:
        print(f"Exception occurred during training: {e}")
        return 0


def objective(trial):
    base_config = deepcopy(config)  # Get fresh copy for each trial
    R = train_model_optuna(trial, base_config, train_data, val_data)
    return R 


# Configuration
config = {
    "gnn": {
        "input_dim": 1,
        "hidden_dim": 128,
        "output_dim": 1,
        "pathway_groups": pathway_groups,
        "layer_modes": [True, False, True],
        "pooling_mode": "pathway",
        "aggr_modes": ["sum","max","mean"]
    },
    "resnet": {
        "hidden_dim": 512,
        "n_layers": 6,
        "dropout": 0.1,
    },
    "drug": {
        "input_dim": 2048,
        "embed_dim": 128
    },
    "optimizer": {
        "learning_rate": 1e-3,
        "clip_norm": 1.0,
        "batch_size": 2,
        "stopping_patience": 10,
    },
    "env": {
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "max_epochs": 50,
        "search_hyperparameters": True  
    }
}

In [7]:
if config["env"]["search_hyperparameters"]:
    study_name = "baseline_model"
    storage_name = f"sqlite:///studies/{study_name}.db"
    unique_study_name = f"baseline_model_{str(uuid.uuid4())[:8]}"  # Random suffix for uniqueness

    study = optuna.create_study(
        study_name=unique_study_name,
        storage=storage_name,
        direction='maximize',
        load_if_exists=False,  # No issue since the study name is always unique
        pruner=optuna.pruners.MedianPruner(n_startup_trials=30, n_warmup_steps=5, interval_steps=5)
    )


    study.optimize(objective, n_trials=40)

    best_config = study.best_params
    print("Best Config: ", best_config)

    config["gnn"].update({
        "hidden_dim": best_config["hidden_dim"],
        "output_dim": best_config["output_dim"],
        "pooling_mode": best_config["pooling_mode"],
        "aggr_modes": best_config["aggr_modes"]
    })
    
    config["resnet"].update({
        "hidden_dim": best_config["hidden_dim_resnet"],
        "n_layers": best_config["n_layers"],
        "dropout": best_config["dropout"]
    })
    
    config["optimizer"].update({
        "learning_rate": best_config["learning_rate"],
        "clip_norm": best_config["clip_norm"],
        "batch_size": best_config["batch_size"]
    })
    
    _, model = train_model(config, train_data, val_data, use_momentum=False)
    
    device = torch.device(config["env"]["device"])
    metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection({
        "R_cellwise_residuals": torchmetrics.PearsonCorrCoef(num_outputs=1),
        "R_cellwise": torchmetrics.PearsonCorrCoef(num_outputs=1),
        "MSE": torchmetrics.MeanSquaredError()
    }))
    metrics.to(device)

    test_dataloader = DataLoader(
        dataset=test_data, 
        batch_size=config["optimizer"]["batch_size"],
        shuffle=True, 
        drop_last=True, 
        collate_fn=custom_collate_fn  
    )

    final_metrics = evaluate_step(model, test_dataloader, metrics, device)
    print("Final Test Metrics: ", final_metrics)

[I 2024-12-10 17:41:56,279] A new study created in RDB with name: baseline_model_0c155cf2
/work/haarscheid/.conda/envs/GNN2/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains ('sum', 'mean', 'max') which is of type tuple.
  warnings.warn(message)
/work/haarscheid/.conda/envs/GNN2/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains ('mean', 'max', 'sum') which is of type tuple.
  warnings.warn(message)
/work/haarscheid/.conda/envs/GNN2/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains ('max', 'sum', 'mean') which is of type tuple.
  warnings.warn(message

Using aggregation mode: ('max', 'max', 'max')
Initializing lazy layers...
Start of forward pass
Exception occurred during training: ModularPathwayConv.propagate() got multiple values for argument 'x'
Using aggregation mode: ('max', 'sum', 'mean')
Initializing lazy layers...
Start of forward pass
Exception occurred during training: ModularPathwayConv.propagate() got multiple values for argument 'x'
Using aggregation mode: ('mean', 'mean', 'mean')


[I 2024-12-10 17:41:57,334] Trial 2 finished with value: 0.0 and parameters: {'hidden_dim': 245, 'output_dim': 210, 'pooling_mode': 'pathway', 'aggr_modes': ['mean', 'mean', 'mean'], 'hidden_dim_resnet': 278, 'n_layers': 3, 'dropout': 0.26562417052632703, 'learning_rate': 8.372016971977575e-06, 'clip_norm': 10.604362593046513, 'batch_size': 2}. Best is trial 0 with value: 0.0.


Initializing lazy layers...
Start of forward pass
Exception occurred during training: ModularPathwayConv.propagate() got multiple values for argument 'x'
Using aggregation mode: ('max', 'max', 'max')


[I 2024-12-10 17:41:57,705] Trial 3 finished with value: 0.0 and parameters: {'hidden_dim': 93, 'output_dim': 110, 'pooling_mode': 'pathway', 'aggr_modes': ['max', 'max', 'max'], 'hidden_dim_resnet': 298, 'n_layers': 6, 'dropout': 0.4623594504277992, 'learning_rate': 2.934509472394252e-06, 'clip_norm': 14.407432911484786, 'batch_size': 2}. Best is trial 0 with value: 0.0.
[I 2024-12-10 17:41:57,862] Trial 4 finished with value: 0.0 and parameters: {'hidden_dim': 369, 'output_dim': 219, 'pooling_mode': 'scalar', 'aggr_modes': ['sum', 'mean', 'max'], 'hidden_dim_resnet': 330, 'n_layers': 4, 'dropout': 0.4955423732248067, 'learning_rate': 5.833730659184285e-06, 'clip_norm': 1.003779716534927, 'batch_size': 4}. Best is trial 0 with value: 0.0.


...

[I 2024-12-10 17:42:10,458] Trial 37 finished with value: 0.0 and parameters: {'hidden_dim': 205, 'output_dim': 224, 'pooling_mode': 'scalar', 'aggr_modes': ['max', 'max', 'max'], 'hidden_dim_resnet': 205, 'n_layers': 3, 'dropout': 0.060541079461341345, 'learning_rate': 4.463753448820818e-06, 'clip_norm': 17.183762846973206, 'batch_size': 2}. Best is trial 0 with value: 0.0.
[I 2024-12-10 17:42:10,661] Trial 38 finished with value: 0.0 and parameters: {'hidden_dim': 65, 'output_dim': 122, 'pooling_mode': 'none', 'aggr_modes': ['sum', 'sum', 'sum'], 'hidden_dim_resnet': 114, 'n_layers': 2, 'dropout': 0.3471753001220098, 'learning_rate': 0.024493364847154968, 'clip_norm': 10.697564970844054, 'batch_size': 1}. Best is trial 0 with value: 0.0.


Using aggregation mode: ('sum', 'sum', 'sum')
Initializing lazy layers...
Start of forward pass
Exception occurred during training: ModularPathwayConv.propagate() got multiple values for argument 'x'
Using aggregation mode: ('sum', 'mean', 'max')


[I 2024-12-10 17:42:11,704] Trial 39 finished with value: 0.0 and parameters: {'hidden_dim': 251, 'output_dim': 240, 'pooling_mode': 'pathway', 'aggr_modes': ['sum', 'mean', 'max'], 'hidden_dim_resnet': 382, 'n_layers': 6, 'dropout': 0.322792067000989, 'learning_rate': 0.00010124947681856569, 'clip_norm': 12.602290044803501, 'batch_size': 5}. Best is trial 0 with value: 0.0.


Initializing lazy layers...
Start of forward pass
Exception occurred during training: ModularPathwayConv.propagate() got multiple values for argument 'x'
Best Config:  {'hidden_dim': 196, 'output_dim': 114, 'pooling_mode': 'scalar', 'aggr_modes': ['max', 'max', 'max'], 'hidden_dim_resnet': 159, 'n_layers': 3, 'dropout': 0.42356644524752696, 'learning_rate': 0.00020552214871559323, 'clip_norm': 13.946054905560707, 'batch_size': 8}


TypeError: train_model() got an unexpected keyword argument 'use_momentum'